In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from tqdm import tqdm


#sns.set(color_codes=True)

In [2]:
# Make proper file names for Windows
# pip install regex

import re

def windows_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [3]:
def plot_graph(plt,start_time, variable):
    plt.savefig(windows_filename(start_time)+'_' + variable+ '.pdf')    

In [49]:
def ftoc(f):
    return (f - 32) * 5.0/9.0

def mphtoms(mph):
    return mph * 0.44704

In [45]:
from pathlib import Path

basePathPatrick = Path("C:/Users/pkastner/Google Drive/Patrick/OutdoorComfort-Biking/Measurements/")
basePathESL = Path("C:/Users/pkastner/Google Drive/Patrick/OutdoorComfort-Biking/Measurements/Remy/_Stations/")
basePathRemy = Path("C:/Users/remym/Dropbox/Cornell 2019-2020/ESL Independent Study/_Stations")
basePathRemy = Path("C:/Users/remym/Dropbox/Cornell 2019-2020/ESL Independent Study/_StationDataPython")
basePathBenj = Path("C:/Users/pkastner/Google Drive/Patrick/OutdoorComfort-Biking/Measurements/Benjamin_StationFile")

loadPath = basePathPatrick

In [ ]:
from hoboreader import HoboReader
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Variable to plot
variable1 = "Wind Speed"
variable2 = "Wind Direction"

#df1[variable].hist()
sns.distplot(df1[variable1], kde=False, fit=stats.gamma);
sns.distplot(df2[variable1], kde=False, fit=stats.gamma);
sns.distplot(df3[variable1], kde=False, fit=stats.gamma);
sns.distplot(df4[variable1], kde=False, fit=stats.gamma);

#sns.jointplot(x=variable1, y=variable2, data=df1);

In [ ]:
from windrose import WindroseAxes
from windrose import WindAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np

def plotWR(ws, wd):
    ax = WindroseAxes.from_ax()
    ax.contourf(wd, ws, bins=np.arange(0, 6, 1),nsector= 36, cmap=cm.hot)
    ax.contour(wd, ws, bins=np.arange(0, 6, 1),nsector= 36, colors='black', lw=1)
    ax.set_legend()
    
def printPDF(ws):
    ax = WindAxes.from_ax()
    bins = np.arange(0, 6 + 1, 0.5)
    bins = bins[1:]
    ax, params = ax.pdf(ws, bins=bins)

In [ ]:


list = [df1, df2, df3, df4]

# Variable to plot
variable1 = "Wind Speed"
variable2 = "Wind Direction"

ws = np.array(df1[variable1]).flatten()
wd = np.array(df1[variable2]).flatten()

for i in tqdm(list):
    ws = np.array(i[variable1]).flatten()
    wd = np.array(i[variable2]).flatten()
    plotWR(ws, wd)
    printPDF(ws)
    
     
    


In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path

      
    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-17 06:00:00-04:00'
end_time = '2019-10-17 19:00:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(15,5)

# Change lineweights
lineweights = 2

# Variable to plot
variable = 'Solar Radiation'

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='blue')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='yellow',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='red',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='magenta',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Global Radiation $[W/m^2]$')
plt.xlabel("Time")
     
# Title
ax.set_title(variable)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path
     
    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-17 06:00:00-04:00'
end_time = '2019-10-17 19:00:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(15,5)

# Change lineweights
lineweights = 2

# Variable to plot
variable = "Wind Speed"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='blue')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='yellow',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='red',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='magenta',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind velocity $[m/s]$')
plt.xlabel("Time")
     
# Title
ax.set_title(variable)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   


#"#","Date Time, GMT-04:00",
#"Temp, °F (LGR S/N: 20702490, SEN S/N: 20496098, LBL: MRT)",
#"Temp, °F (LGR S/N: 20702490, SEN S/N: 20698454, LBL: Drybulb)",
#"RH, % (LGR S/N: 20702490, SEN S/N: 20698454)",
#"Solar Radiation, W/m² (LGR S/N: 20702490, SEN S/N: 20702165, LBL: Rad)",
#"Wind Speed, mph (LGR S/N: 20702490, SEN S/N: 20706788, LBL: Wind)",
#"Gust Speed, mph (LGR S/N: 20702490, SEN S/N: 20706788)",
#"Wind Direction, ø (LGR S/N: 20702490, SEN S/N: 20706788)",
#"DewPt, °F (LGR S/N: 20702490, SEN S/N: 20698454)"
    

S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df4=S4.get_dataframe()

# Select time

start_time = '2019-09-01 06:00:00-04:00'
end_time = '2019-12-15 19:00:00-04:00'


df4_ = df4.loc[start_time:end_time]

# Resize figure
custom_figsize=(25,5)

# Change lineweights
lineweights = 2


# Variable to plot
variable = "Temp"
variable2 = "Temp"


test= df4_[variable].subtract(df4_[variable2], fill_value=0)


print(test)


# Plot two dataframes into one plot
#ax = plt.plot(diff,lw = lineweights, figsize=custom_figsize,color='blue')
#
## Subset Time Series By Dates Python Using Pandas
##https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/
#
## Legend
#ax.legend([
#"DL1 - Stone Garden - Tjaden Hall", 
#"DL2 - PSB Parking Lot",
#"DL3 - Olin Library",
#"DL4 - Milstein Hall Roof "  
#],loc='upper left', frameon=False)
#
## Fix xticks
#
#years = mdates.YearLocator()   # every year
#months = mdates.MonthLocator()  # every month
#days = mdates.DayLocator()  # every minute
#hours = mdates.HourLocator()  # every minute
#minutes = mdates.MinuteLocator()  # every minute
#
#days_fmt = mdates.DateFormatter('%d')
#hours_fmt = mdates.DateFormatter('%H:00')
#
## format the ticks
#ax.xaxis.set_major_locator(hours)
#ax.xaxis.set_major_formatter(hours_fmt)
#ax.xaxis.set_minor_locator(hours)
#
##Labels
#plt.ylabel('Wind velocity $[m/s]$')
#plt.xlabel("Time")
#     
## Title
#ax.set_title(variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path

    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(loadPath / "20191101_11-04_DL1.csv") 
S2=HoboReader(loadPath / "20191101-10-50_DL2.csv")
S3=HoboReader(loadPath / "20191115-11-08_DL3.csv")
S4=HoboReader(loadPath / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-17 06:00:00-04:00'
end_time = '2019-10-17 19:00:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(15,5)

# Change lineweights
lineweights = 2

# Variable to plot
variable = "Wind Direction"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='blue')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='yellow',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='red',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='magenta',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind direction $[°]$')
plt.xlabel("Time")
     
# Title
ax.set_title(variable)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
# conda install -c https://conda.anaconda.org/ioos geopandas
# conda install -c conda-forge shapely
# conda install -c conda-forge descartes

import geopandas as gpd
from shapely.geometry import Point, Polygon
import descartes

#final_crs = {'init': 'epsg:28992'}

buildings = gpd.read_file('Ithaca/map/buildings-polygon.shp')#.to_crs(final_crs)
roads = gpd.read_file('Ithaca/map/roads-line.shp')#.to_crs(final_crs)
landcover = gpd.read_file('Ithaca/map/landcover-polygon.shp')#.to_crs(final_crs)
water = gpd.read_file('Ithaca/map/water_areas-polygon.shp')#.to_crs(final_crs)
waterlines = gpd.read_file('Ithaca/map/water_lines-line.shp')#.to_crs(final_crs)

fig,ax = plt.subplots(figsize= (10,10))

roads.plot(ax=ax, color= "grey")
landcover.plot(ax=ax, color= "green", alpha=0.03)
water.plot(ax=ax, color= "blue", alpha=0.1)
waterlines.plot(ax=ax, color= "blue", alpha=0.1)
buildings.plot(ax=ax, color= "black")

minx, miny, maxx, maxy = -76.4900, 42.443484, -76.475089, 42.4555
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

DL1= Point((-76.485089,42.450960))
DL2= Point((-76.481194,42.450536))
DL3= Point((-76.484268,42.448039))
DL4= Point((-76.483743,42.451245))

gdf_stations = gpd.GeoSeries([DL1,DL2,DL3,DL4])
gdf_stations.plot(ax=ax, markersize= 50, color= "red", marker = "o")

# Annotate with names

stations = [
"DL1", 
"DL2",
"DL3",
"DL4"  
]
xx = 0.0003
yy = 0.0001

for i, geo in gdf_stations.centroid.iteritems():
    ax.annotate(s=stations[i], xy=[geo.x+xx, geo.y-yy], color="red" )


# Save PDF
plt.savefig('Map.pdf')

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick / "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick / "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick / "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 1.5

# Variable to plot
variable = "Wind Direction"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind direction $[°]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


      
    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick/ "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick/ "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick/ "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick/ "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 1.5

# Variable to plot
variable = "Wind Speed"

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Wind Speed $[MPH]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick / "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick / "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick / "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 2

# Variable to plot
variable = 'Solar Radiation'

# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Global Radiation $[W/m^2]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path


    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
  
    
S1=HoboReader(basePathPatrick / "20191101_11-04_DL1.csv") 
S2=HoboReader(basePathPatrick / "20191101-10-50_DL2.csv")
S3=HoboReader(basePathPatrick / "20191115-11-08_DL3.csv")
S4=HoboReader(basePathPatrick / "20191101-12-58_DL4.csv")


df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()

# Select time

start_time = '2019-10-31 00:00:00-04:00'
end_time = '2019-11-01 23:59:00-04:00'

df1_ = df1.loc[start_time:end_time]
df2_ = df2.loc[start_time:end_time]
df3_ = df3.loc[start_time:end_time]
df4_ = df4.loc[start_time:end_time]

#df1.index

# Print Headers
#print(df1.columns.values)


# Resize figure
custom_figsize=(40,15)

# Change lineweights
lineweights = 1.5

# Variable to plot
variable = "Temp"

# Translate to °C

df1_[variable] = df1_[variable].apply(lambda x: f2c(x), axis=1)
df2_[variable] = df2_[variable].apply(lambda x: f2c(x), axis=1)
df3_[variable] = df3_[variable].apply(lambda x: f2c(x), axis=1)
df4_[variable] = df4_[variable].apply(lambda x: f2c(x), axis=1)


# Plot two dataframes into one plot
ax = df1_.plot(y=[variable],lw = lineweights, figsize=custom_figsize,color='red')
df3_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='blue',ax=ax)
df2_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='green',ax=ax)
df4_.plot(     y=[variable],lw = lineweights, figsize=custom_figsize,color='aqua',ax=ax)

# Subset Time Series By Dates Python Using Pandas
#https://www.earthdatascience.org/courses/earth-analytics-python/use-time-series-data-in-python/subset-time-series-data-python/


# Legend
ax.legend([
"DL1 - Stone Garden - Tjaden Hall", 
"DL2 - PSB Parking Lot",
"DL3 - Olin Library",
"DL4 - Milstein Hall Roof "  
],loc='upper left', frameon=False)


# Fix xticks

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
days = mdates.DayLocator()  # every minute
hours = mdates.HourLocator()  # every minute
minutes = mdates.MinuteLocator()  # every minute

days_fmt = mdates.DateFormatter('%d')
hours_fmt = mdates.DateFormatter('%H:00')

# format the ticks
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(hours_fmt)
ax.xaxis.set_minor_locator(hours)

#Labels
plt.ylabel('Temp $[°C]$', fontname="Consolas", fontsize=20)
plt.xlabel("Time", fontname="Consolas", fontsize=20)
     
# Title
ax.set_title(variable, fontname="Consolas", fontsize=28)

# Save PDF
plot_graph(plt, start_time, variable)

In [ ]:
# conda install -c https://conda.anaconda.org/ioos geopandas
# conda install -c conda-forge shapely
# conda install -c conda-forge descartes

import geopandas as gpd
from shapely.geometry import Point, Polygon
import descartes

#final_crs = {'init': 'epsg:28992'}

buildings = gpd.read_file('Ithaca/map/buildings-polygon.shp')#.to_crs(final_crs)
roads = gpd.read_file('Ithaca/map/roads-line.shp')#.to_crs(final_crs)
landcover = gpd.read_file('Ithaca/map/landcover-polygon.shp')#.to_crs(final_crs)
water = gpd.read_file('Ithaca/map/water_areas-polygon.shp')#.to_crs(final_crs)
waterlines = gpd.read_file('Ithaca/map/water_lines-line.shp')#.to_crs(final_crs)

fig,ax = plt.subplots(figsize= (10,10))

roads.plot(ax=ax, color= "grey")
landcover.plot(ax=ax, color= "green", alpha=0.03)
water.plot(ax=ax, color= "blue", alpha=0.1)
waterlines.plot(ax=ax, color= "blue", alpha=0.1)
buildings.plot(ax=ax, color= "black")

minx, miny, maxx, maxy = -76.4900, 42.443484, -76.475089, 42.4555
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

DL1= Point((-76.485089,42.450960))
DL2= Point((-76.481194,42.450536))
DL3= Point((-76.484268,42.448039))
DL4= Point((-76.483743,42.451245))

gdf_stations = gpd.GeoSeries([DL1,DL2,DL3,DL4])
gdf_stations.plot(ax=ax, markersize= 50, color= "red", marker = "o")

# Annotate with names

stations = [
"DL1", 
"DL2",
"DL3",
"DL4"  
]
xx = 0.0003
yy = 0.0001

for i, geo in gdf_stations.centroid.iteritems():
    ax.annotate(s=stations[i], xy=[geo.x+xx, geo.y-yy], color="red" )


# Save PDF
plt.savefig('Map.pdf')

In [1]:
# Merge all recorded data



from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
%matplotlib inline

pd.set_option('mode.chained_assignment',None)

# Set up figure size and DPI for screen demo
plt.rcParams['figure.figsize'] = (6,4)
plt.rcParams['figure.dpi'] = 150





#loadPath = Path(r"C:\Users\remym\Dropbox\ESL\CSV Merged Data")
#loadPath = Path(r"C:\Users\pkastner\Google Drive\Patrick\OutdoorComfort-Biking\Measurements")

#S1=HoboReader(loadPath / "DL1_Stone_Garden_Tjaden_Hall.csv") 
#S2=HoboReader(loadPath / "DL2_PSB_Parking_Lot.csv")
#S3=HoboReader(loadPath / "DL3_Olin_Library.csv")
#S4=HoboReader(loadPath / "DL4_Milstein_Hall_Roof.csv")
#S5=HoboReader(LoadPath / "DL5_Game_Farm_Road_DST20192020.csv")

    
#DL1	Stone Garden - Tjaden Hall
#DL2	PSB Parking Lot
#DL3	Olin Library
#DL4	Milstein Hall Roof   
#DL5	Game Farm Road   
  
    
S1=HoboReader("DL1_Stone_Garden_Tjaden_Hall.csv") 
S2=HoboReader("DL2_PSB_Parking_Lot.csv")
S3=HoboReader("DL3_Olin_Library.csv")
S4=HoboReader("DL4_Milstein_Hall_Roof.csv")
S5=HoboReader("DL5_Game_Farm_Road_DST20192020.csv")



df1=S1.get_dataframe()
df2=S2.get_dataframe()
df3=S3.get_dataframe()
df4=S4.get_dataframe()
df5=S5.get_dataframe()

df5=df5.shift(periods=(-1), fill_value=0 )

def ftoc(f):
    return (f - 32) * 5.0/9.0

def mphtoms(mph):
    return mph * 0.44704

def f(variable_id,d, m,y):
            
    start_time = str(y)+'-'+str(m)+'-'+str(d)+' 07:00:00-00:00'
    end_time =   str(y)+'-'+str(m)+'-'+str(d)+' 23:00:00-00:00'
    
   
    df1_ = df1.loc[start_time:end_time]
    df2_ = df2.loc[start_time:end_time]
    df3_ = df3.loc[start_time:end_time]
    df4_ = df4.loc[start_time:end_time]
    df5_ = df5.loc[start_time:end_time]


    # Translate to °C

    df1_["Temp"] = df1_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df2_["Temp"] = df2_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df3_["Temp"] = df3_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df4_["Temp"] = df4_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df5_["Temp"] = df5_["Temp"].apply(lambda x: ftoc(x), axis=1)
    
    df1_["MRT"] = df1_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df2_["MRT"] = df2_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df3_["MRT"] = df3_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df4_["MRT"] = df4_["MRT"].apply(lambda x: ftoc(x), axis=1)
    #df5_["MRT"] = df5_["MRT"].apply(lambda x: ftoc(x), axis=1)
    
    # Translate to m/s
    
    df1_["Wind Speed"] = df1_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df2_["Wind Speed"] = df2_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df3_["Wind Speed"] = df3_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df4_["Wind Speed"] = df4_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df5_["Wind Speed"] = df5_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    
    df1_["Gust Speed"] = df1_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df2_["Gust Speed"] = df2_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df3_["Gust Speed"] = df3_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df4_["Gust Speed"] = df4_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df5_["Gust Speed"] = df5_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    
    
    #df1.index
    
    # Print Headers
    #print(df1.columns.values)
    
    
    # Resize figure
    custom_figsize=(20,10)
    
    # Change lineweights
    lineweights = 1.5
    
    

    
    variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPt" ]  
    
    
    
    variable = variables[variable_id]
    
    # Plot both temp and mrt when temp is selected
    
    if variable == "MRT" or variable == "DewPt" or variable == "Gust Speed" :
        ax = df1_.plot(y = variable, lw=1, figsize=custom_figsize,color='red')
        df2_.plot( y = variable, lw=1, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable, lw=1, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable, lw=1, figsize=custom_figsize,color='aqua',ax=ax)  
    
    elif variable == "Temp":
        # Plot two dataframes into one plot
        ax = df1_.plot(y = variable, lw=2, figsize=custom_figsize,color='red')
        df2_.plot( y = variable,    lw=2, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable,     lw=2, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable,     lw=2, figsize=custom_figsize,color='aqua',ax=ax)
        df5_.plot(y = variable,     lw=3.5, figsize=custom_figsize,color='black',ax=ax)
        
        ax = df1_.plot(y = "MRT", lw=1, figsize=custom_figsize,style = ['--'],color='red', ax=ax)
        df2_.plot( y = "MRT",     lw=1, figsize=custom_figsize, style = ['--'],color='blue',ax=ax)
        df3_.plot(y = "MRT",      lw=1, figsize=custom_figsize, style = ['--'],color='green',ax=ax)
        df4_.plot(y = "MRT",      lw=1, figsize=custom_figsize, style = ['--'],color='aqua',ax=ax)      
                  
    else:  
        # Plot two dataframes into one plot
        ax = df1_.plot(y = variable, lw=1, figsize=custom_figsize,color='red')
        df2_.plot( y = variable, lw=1, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable, lw=1, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable, lw=1, figsize=custom_figsize,color='aqua',ax=ax)
        df5_.plot(y = variable, lw=2, figsize=custom_figsize,color='black',ax=ax)
    
    # Legend
    ax.legend([
    "DL1 - Stone Garden - Tjaden Hall", 
    "DL2 - PSB Parking Lot",
    "DL3 - Olin Library",
    "DL4 - Milstein Hall Roof ",
    "DL5 - Game Farm Road"
    
    ],loc='upper left', frameon=False)
    
    
    # Fix xticks
    
    years = mdates.YearLocator()   # every year
    months = mdates.MonthLocator()  # every month
    days = mdates.DayLocator()  # every minute
    hours = mdates.HourLocator()  # every minute
    minutes = mdates.MinuteLocator()  # every minute
    
    days_fmt = mdates.DateFormatter('%d')
    hours_fmt = mdates.DateFormatter('%H:00')
    
    # format the ticks
    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(hours_fmt)
    ax.xaxis.set_minor_locator(hours)
    
    #Labels
    plt.ylabel(variable, fontname="Consolas", fontsize=20)
    plt.xlabel("UTC Time", fontname="Consolas", fontsize=20)
         
    # Title
    ax.set_title(str(start_time.split(' ')[0]+" "+variable), fontname="Consolas", fontsize=28)
    
    # Save PDF
    #plot_graph(plt, start_time, variable)

    
# Variable to plot
#1 = "MRT °C"
#2 = "Temp °C"
#3 = "RH"
#4 = "Solar Radiation"
#5 = "Wind Speed"
#6 = "Gust Speed"
#7 = "Wind Direction"
#8 = "DewPt"
    
    
interact(f,variable_id=(1,8,1), d=(1,31,1), m=(1,12,1), y = (2019,2020,1))










interactive(children=(IntSlider(value=4, description='variable_id', max=8, min=1), IntSlider(value=16, descrip…

<function __main__.f(variable_id, d, m, y)>

In [87]:
display(df5)

title,#,Date Time,Temp,RH,Wind Speed,Wind Direction,Solar Radiation
timezone_str,NaN,GMT-05:00,NaN,NaN,NaN,NaN,NaN
units,NaN,NaN,Â°F,%,mph,Ã¸,W/mÂ²
logger_serial_number,NaN,NaN,20098520,21098500,20198500,20198500,20034090
sensor_serial_number,NaN,NaN,10762331,10263331,21706789,21706789,23073988
datetimes,,,,,,,
2019-05-27 00:00:00-05:00,2,05/27/19 01:00:00 AM,54.37,93.70,3.32,116.7,0.00
2019-05-27 01:00:00-05:00,3,05/27/19 02:00:00 AM,52.05,94.70,3.16,111.7,0.00
2019-05-27 02:00:00-05:00,4,05/27/19 03:00:00 AM,49.89,96.20,3.50,121.4,0.00
2019-05-27 03:00:00-05:00,5,05/27/19 04:00:00 AM,49.01,96.70,2.90,109.8,0.00
2019-05-27 04:00:00-05:00,6,05/27/19 05:00:00 AM,47.77,97.30,3.28,95.8,0.00


In [ ]:
S5=HoboReader("DL5_Game_Farm_Road_2.csv")

df5=S5.get_dataframe()
df5



In [73]:
import pandas as pd
from pandas import read_csv
import csv
import unittest
from pprint import pprint
import pandas as pd
from setuptools import setup, find_packages
from codecs import open
from os import path

loadPath = Path(r"C:\Users\remym\Dropbox\ESL\CSV Merged Data")

read_csv =("DL5_Game_Farm_Road_DST20192020.csv")

# OBJECT CREATION
    
def test_epw___init__(self):
        a=epw()
        self.assertIsInstance(a,epw)
        print(a)


def test_epw_read(self):
        a=epw()
        a.read(DL5_Game_Farm_Road_DST20192020.csv)
        print(a.headers)
        print(a.dataframe.columns)
        print(a.dataframe[['Year', 'Month', 'Day', 'Hour', 'Minute','Dry Bulb Temperature']].head())
        

def test_modify(self):
        a=epw()
        a.read(DL5_Game_Farm_Road_DST20192020.csv)
        
        a.headers['LOCATION'][0]='Ithaca_NY'
        print(a.headers['LOCATION'])
        
        a.dataframe['Dry Bulb Temperature']=old_file['Dry Bulb Temperature']+1.0
        print(a.dataframe[['Year', 'Month', 'Day', 'Hour', 'Minute','Dry Bulb Temperature']].head())

a.write=("new_epw_file")
path= (r"C:\Users\remym\Documents\GitHub\HoboPlot")




# Direct/Diffuse split


We need the extra terrestrial radiation for the diffuse part which we can get with "get_extra_radiation()".

https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.get_extra_radiation.html?highlight=dni_extra%20

From there, we can use "reindl()" to calculate the split.

https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.reindl.html?highlight=reindl


https://pvpmc.sandia.gov/modeling-steps/1-weather-design-inputs/plane-of-array-poa-irradiance/


In [36]:
#!pip install pvlib
import pvlib
# https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.reindl.html?highlight=reindl#pvlib.irradiance.reindl

# conda install -c pvlib pvlib
import pandas as pd

# Create date series using date_range() function
date_series = pd.date_range('01/01/2019', periods = 8760, freq ='H')
#print(date_series)

# altitude GFR Station
altitude = 294

# lat, lon taken from GFR
lon = 42.449
lat =  -76.449

pos = pvlib.solarposition.get_solarposition(date_series,lat, lon)
pos


,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
2019-01-01 00:00:00,77.260014,77.331949,12.739986,12.668051,141.183660,-3.195874
2019-01-01 01:00:00,74.762141,74.822449,15.237859,15.177551,127.206717,-3.215687
2019-01-01 02:00:00,71.746085,71.796274,18.253915,18.203726,113.152209,-3.235491
2019-01-01 03:00:00,68.389636,68.431648,21.610364,21.568352,98.919467,-3.255288
2019-01-01 04:00:00,64.896566,64.932217,25.103434,25.067783,84.379882,-3.275076
...,...,...,...,...,...,...
2019-12-31 19:00:00,78.337861,78.416184,11.662139,11.583816,211.035498,-2.982273
2019-12-31 20:00:00,79.759268,79.847817,10.240732,10.152183,197.068349,-3.002181
2019-12-31 21:00:00,80.370498,80.464252,9.629502,9.535748,183.085328,-3.022080
2019-12-31 22:00:00,80.136801,80.228499,9.863199,9.771501,169.098117,-3.041970


In [31]:
# this is what we want
# dni (float or Series) – Direct Normal Irradiance
# dhi (float or Series) – Diffuse horizontal irradiance

# this is what we have
# ghi (float or Series) – Global horizontal irradiance

## Extra terrestrial radiation

albedo = 0.25 # grass
surface_tilt = 1 # flat surface
ghi  = np.array([200,300,400,500])





array([0.00380762, 0.00571143, 0.00761524, 0.00951905])

In [ ]:
# Global Horizontal (GHI) = Direct Normal (DNI) X cos(θ) + Diffuse Horizontal (DHI)











###

datetime_or_doy = 1  # (numeric, array, date, datetime, Timestamp, DatetimeIndex) – Day of year, array of days of year, or datetime-like object

# (string, default 'spencer') – The method by which the ET radiation should be calculated. Options include 'pyephem', 'spencer', 'asce', 'nrel'.
epoch_year = 2019    # (int, default 2014) – The year in which a day of year input will be calculated. Only applies to day of year input used with the pyephem or nrel methods.

dni_extra= pvlib.irradiance.get_extra_radiation(datetime_or_doy, solar_constant=1366.1, method='nrel', epoch_year=2014)
print(dni_extra)


## Reindl Split

surface_tilt = 0  # (numeric)– Surface tilt angles in decimal degrees. The tilt angle is defined as degrees from horizontal (e.g. surface facing up = 0, surface facing horizon = 90)
surface_azimuth = 0  # (numeric) – Surface azimuth angles in decimal degrees. The azimuth convention is defined as degrees east of north (e.g. North = 0, South=180 East = 90, West = 270).
dhi = 200  #(numeric) – diffuse horizontal irradiance in W/m^2.
dni = 200  #(numeric) – direct normal irradiance in W/m^2.
ghi = 200  #(numeric) – Global irradiance in W/m^2.
#dni_extra = 200  #(numeric) – Extraterrestrial normal irradiance in W/m^2.
solar_zenith = 10  #(numeric) – Apparent (refraction-corrected) zenith angles in decimal degrees.
solar_azimuth = 20  #(numeric) – Sun azimuth angles in decimal degrees. The azimuth convention is defined as degrees east of north (e.g. North = 0, East = 90, West = 270).




# (numeric) – The sky diffuse component of the solar radiation.
poa_sky_diffuse = pvlib.irradiance.reindl(surface_tilt, surface_azimuth, dhi,
                                          dni, ghi, dni_extra, solar_zenith,
                                          solar_azimuth)

print(poa_sky_diffuse)
